In [27]:
from IPython.core import magic_arguments
from IPython.core.magic import line_magic, cell_magic, line_cell_magic, Magics, magics_class
from IPython.display import HTML, display
import boto3 
import pandas as pd
 
@magics_class
class TestMagics(Magics):
    @cell_magic
    def hello(self, line='', cell=None):
        print('hello ' + cell)
 
    @line_magic
    def hi(self, line):
        print('hi ' + line)
        
    @line_magic
    def db(self, line):
        ec2client = boto3.client('ec2')
        response = ec2client.describe_instances()
        reservations = response.get('Reservations')
        instances = []
        for reservation in reservations:
            reservationInstances = reservation.get('Instances')
            for inst in reservationInstances:
                instances.append(inst)
        
        instancesFormatted = []
        
        for instance in instances:
            tags = instance.get('Tags', [])
            name = ''
            for tag in tags:
                tagKey = tag.get('Key', '')
                if tagKey == 'Name':
                    name = tag['Value']

            placement = instance['Placement']
            availabilityZone = placement['AvailabilityZone']

            state = instance['State']
            stateName = state.get('Name', '')

            launchTime = instance.get('LaunchTime', '')

            if len(name) > 20:
                name = name[:20] + '...'

            formatInst = {
                'Name': name,
                'Instance Id': instance.get('InstanceId', ''),
                'Instance Type': instance.get('InstanceType', ''),
                'Availability Zone': availabilityZone,
                'State': stateName,
                'Key Name': instance.get('KeyName', ''),
                'Launch Time': launchTime,
            }
            instancesFormatted.append(formatInst)
        display(HTML(
        '<table><tr>{}</tr></table>'.format(
           '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(key) for key in row)) for row in instancesFormatted)
       )
        ))
 
ip = get_ipython()
ip.register_magics(TestMagics)

In [28]:
%db

Name,Instance Id,Instance Type,Availability Zone,State,Key Name,Launch Time


[{'Availability Zone': 'us-west-1c',
  'Instance Id': 'i-0c61ebe4a24d26901',
  'Instance Type': 't2.micro',
  'Key Name': 'key_pair_guppi',
  'Launch Time': datetime.datetime(2019, 2, 14, 23, 33, 57, tzinfo=tzutc()),
  'Name': '',
  'State': 'terminated'}]